In [ ]:
import optuna
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
from torchvision import models
import torch.nn as nn
import torch.optim as optim

# Data pre-processing transformations
transform = transforms.Compose([
    transforms.Lambda(lambda img: img.convert('RGB')),  # Convert grayscale to RGB
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
])

# Download Caltech101 dataset
dataset = datasets.Caltech101(root='./data', download=True, transform=transform)

# Split into train, validation, and test sets
train_size = int(0.7 * len(dataset))
val_size = int(0.2 * len(dataset))
test_size = len(dataset) - train_size - val_size

train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

# Initialize data loaders
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)

# Define the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def objective(trial):
    # Hyperparameter to tune
    lr = trial.suggest_float('lr', 1e-3, 4e-3, log=True)

    # Load pre-trained VGG-19 with batch normalization
    model = models.vgg19_bn(weights='DEFAULT')
    num_classes = 102
    model.classifier[6] = nn.Linear(model.classifier[6].in_features, num_classes)
    model = model.to(device)

    # Loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9)

    # Train and validate the model
    num_epochs = 5
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

    # Validation phase
    model.eval()
    val_running_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

    val_loss = val_running_loss / len(val_loader)
    val_accuracy = 100 * correct / total
    return val_loss

# Optuna study
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=50)

# Results
print('Number of finished trials: ', len(study.trials))
print('Best trial:')
trial = study.best_trial

print('Value: ', trial.value)
print('Params: ')
for key, value in trial.params.items():
    print(f'    {key}: {value}')

[I 2023-10-17 18:10:33,250] A new study created in memory with name: no-name-8d27d2dc-2d74-4654-aba7-e61372ee90dd


Files already downloaded and verified


[I 2023-10-17 18:13:12,382] Trial 0 finished with value: 0.1702079809022377 and parameters: {'lr': 0.003882602293918509}. Best is trial 0 with value: 0.1702079809022377.
[I 2023-10-17 18:16:25,214] Trial 1 finished with value: 0.19835988171897562 and parameters: {'lr': 0.0010432125870407174}. Best is trial 0 with value: 0.1702079809022377.
